In [ ]:

import argparse
import json
from PIL import Image
import numpy as np
import torch
import torch.nn.functional as F
import random
import os

from diffusers import DDIMScheduler

from models.p2p.inversion import DirectInversion
from models.masactrl.diffuser_utils import MasaCtrlPipeline
from models.masactrl.masactrl_utils import AttentionBase
from models.masactrl.masactrl_utils import regiter_attention_editor_diffusers
from models.masactrl.masactrl import MutualSelfAttentionControl
from utils.utils import load_512,txt_draw

from torchvision.io import read_image
from run_editing_masactrl import MasaCtrlEditor



def mask_decode(encoded_mask,image_shape=[512,512]):
    length=image_shape[0]*image_shape[1]
    mask_array=np.zeros((length,))
    
    for i in range(0,len(encoded_mask),2):
        splice_len=min(encoded_mask[i+1],length-encoded_mask[i])
        for j in range(splice_len):
            mask_array[encoded_mask[i]+j]=1
            
    mask_array=mask_array.reshape(image_shape[0], image_shape[1])
    # to avoid annotation errors in boundary
    mask_array[0,:]=1
    mask_array[-1,:]=1
    mask_array[:,0]=1
    mask_array[:,-1]=1
            
    return mask_array


def setup_seed(seed=1234):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
masactrl_editor=MasaCtrlEditor(["ddim+masactrl","directinversion+masactrl"], torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') )

In [ ]:
# ddim+masactrl

edited_image = masactrl_editor("ddim+masactrl",
                    image_path="scripts/example_cake.jpg",
                    prompt_src="a round cake with orange frosting on a wooden plate",
                    prompt_tar="a square cake with orange frosting on a wooden plate",
                    guidance_scale=7.5,
                    step=4,
                    layper=10
                    )

print("null-text-inversion+masactrl")
display(edited_image)

In [ ]:
# directinversion(ours)+p2p

edited_image = masactrl_editor("directinversion+masactrl",
                    image_path="scripts/example_cake.jpg",
                    prompt_src="a round cake with orange frosting on a wooden plate",
                    prompt_tar="a square cake with orange frosting on a wooden plate",
                    guidance_scale=7.5,
                    step=4,
                    layper=10
                    )

print("directinversion(ours)+masactrl")
display(edited_image)

In [ ]:
# ddim+masactrl

edited_image = masactrl_editor("ddim+masactrl",
                    image_path="scripts/example_cat.jpg",
                    prompt_src="a cat sitting on a table with a green eyes",
                    prompt_tar="a watercolor of a cat sitting on a table with a green eyes",
                    guidance_scale=7.5,
                    step=4,
                    layper=10
                    )

print("null-text-inversion+masactrl")
display(edited_image)

In [ ]:
# directinversion+masactrl

edited_image = masactrl_editor("directinversion+masactrl",
                    image_path="scripts/example_cat.jpg",
                    prompt_src="a cat sitting on a table with a green eyes",
                    prompt_tar="a watercolor of a cat sitting on a table with a green eyes",
                    guidance_scale=7.5,
                    step=4,
                    layper=10
                    )

print("directinversion(ours)+masactrl")
display(edited_image)